In [6]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = HuggingFacePipeline.from_model_id(
    model_id="openai-community/gpt2",
    task="text-generation",
    pipeline_kwargs={
        "max_new_tokens": 50,
    },
)

chain = prompt | llm

result = chain.invoke({"word": "potato"})
print(result)

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A potato is a tiny, white plastic bag that floats on the surface of any liquid in the food you're eating. This is not a big deal, just a little bag that hangs on the top with some weight and water, and can go by many times as a
